In [52]:
import modin.pandas as pd
import time
import json
import os

In [63]:
dir_jobs = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\2. Filtered preprocessed data\\2018\\"
dir_countries = "C:\\Users\\eshad\\OneDrive\\Desktop\\TU Delft\\Thesis\\Web Scrape PoC\\Data\\3. Filtered preprocessed data with country\\"

In [64]:
# Function to extract company ID and LinkedIn link
def extract_company_info(company_detail):
    try:
        company_data = json.loads(company_detail.replace("'", '"'))
        if 'com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany' in company_data:
            company_info = company_data['com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany']
        elif 'com.linkedin.voyager.jobs.JobPostingCompanyName' in company_data:
            company_info = company_data['com.linkedin.voyager.jobs.JobPostingCompanyName']
        # print(company_info['company'])
        if 'company' in company_info:
            company_id = company_info['company'].split(':')[-1]
        elif 'companyResolutionResult' in company_info:
            company_id = company_info['companyResolutionResult']['entityUrn'].split(':')[-1]
        elif 'companyName' in company_info:
            company_id = company_info['companyName']
        return company_id
    except (KeyError, ValueError, TypeError) as e:
        return "no company ID"

In [65]:
dataframes = []
# Read all files in the directory
for file_name in os.listdir(dir_jobs):
    # if file_name.startswith('2018_'): 
        print(file_name)
        file_path = os.path.join(dir_jobs, file_name)
        tmp_df = pd.read_csv(file_path)
        dataframes.append(tmp_df)
df = pd.concat(dataframes)
print(len(df))
df.head()

2018_backup_29_mar.csv
2018_backup_3_16_mar.csv
20746


,Unnamed: 0,jobPostingId,companyDetails,title,description,workRemoteAllowed,formattedLocation,work_model,soc_code
0,218,589566654,{'com.linkedin.voyager.deco.jobs.web.shared.We...,VOIP Systems Administrator,VOIP Systems Administrator \n The Texas Associ...,False,"Austin, TX",NaN,313.0
1,222,589565742,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Engineer,Who We Are\n\nWeâre not only the largest onl...,False,"Detroit, MI",NaN,524.0
2,313,589522590,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Senior Front-End Web Developer - (180001FD),The Senior Front-End Web Developer position bu...,False,"Cleveland, OH",NaN,213.0
3,339,589516170,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Analyst - Healthcare Payer,At Perficient youâll deliver mission-critica...,False,"Miami, FL",NaN,242.0
4,361,589515078,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Analyst - Healthcare Payer,At Perficient youâll deliver mission-critica...,False,"Little Rock, AR",NaN,242.0


In [66]:
df_no_na = df.dropna(how='all')
len(df_no_na)

20746

In [67]:
df_no_dupes = df_no_na.drop_duplicates(keep='first')
len(df_no_dupes)

19224

In [68]:
country_dataframes = []
# Read all files in the directory
for file_name in os.listdir(dir_countries):
    if file_name.startswith('locations_'): 
        print(file_name)
        file_path = os.path.join(dir_countries, file_name)
        tmp_df = pd.read_csv(file_path, usecols = ['formattedLocation', 'country'])
        country_dataframes.append(tmp_df)
country_df = pd.concat(country_dataframes)
print(len(country_df))
country_df = country_df.dropna()
country_df = country_df.drop_duplicates()
print(len(country_df))
country_df.head()

locations_and_countries.csv
locations_and_countries_batch_2.csv
locations_and_countries_batch_3.csv
22976
22614


,formattedLocation,country
0,Israel,Israel
1,United States,United States of America
2,"Holmdel, NJ",United States of America
3,"Washington, DC",United States of America
4,"Manhattan Beach, CA",United States of America


In [69]:
df_with_countries = df_no_dupes.merge(country_df, how='left', on='formattedLocation')
print(len(df_with_countries))
df_with_countries.head()

19224


,Unnamed: 0,jobPostingId,companyDetails,title,description,workRemoteAllowed,formattedLocation,work_model,soc_code,country
0,218,589566654,{'com.linkedin.voyager.deco.jobs.web.shared.We...,VOIP Systems Administrator,VOIP Systems Administrator \n The Texas Associ...,False,"Austin, TX",NaN,313.0,United States of America
1,222,589565742,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Engineer,Who We Are\n\nWeâre not only the largest onl...,False,"Detroit, MI",NaN,524.0,United States of America
2,313,589522590,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Senior Front-End Web Developer - (180001FD),The Senior Front-End Web Developer position bu...,False,"Cleveland, OH",NaN,213.0,United States of America
3,339,589516170,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Analyst - Healthcare Payer,At Perficient youâll deliver mission-critica...,False,"Miami, FL",NaN,242.0,United States of America
4,361,589515078,{'com.linkedin.voyager.deco.jobs.web.shared.We...,Data Analyst - Healthcare Payer,At Perficient youâll deliver mission-critica...,False,"Little Rock, AR",NaN,242.0,United States of America


In [70]:
df_with_countries.to_csv(os.path.join(dir_countries, "all_data_with_countries", "all_data_with_countries_2018.csv"))